In [1]:
## アヤメの種類を学習するCNNの実装
## ロジスティック回帰(線形分割)をchainerで実装
import numpy as np
from chainer import cuda, Function, gradient_check,\
    Variable, optimizers, serializers, utils
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L

In [2]:
from sklearn import datasets

## アヤメに関する4次元x150個のデータ
iris = datasets.load_iris()
## 入力データ
X = iris.data.astype(np.float32)
## 訓練データ(出力を種類noから確率に変更)
Y = iris.target.astype(np.int32)
## 入力データサイズ
N = Y.size

print ("shape of iris.data : ", iris.data.shape)

shape of iris.data :  (150, 4)


In [3]:
## アヤメは3種類に分類されるので訓練データ3N次元
Y2 = np.zeros(3*N).reshape(N,3).astype(np.float32)
for i in range(N):
## 正解の種類は1.0, それ以外の2種類は0.0とする
    Y2[i,Y[i]] = 1.0

In [4]:
## 奇数noデータを訓練データ,偶数noデータを検証用に設定
index = np.arange(N)
xtrain =  X[index[index % 2 != 0],:]
ytrain = Y2[index[index % 2 != 0],:]
xtest  =  X[index[index % 2 == 0],:]
yans   =  Y[index[index % 2 == 0]]

In [5]:
## ロジスティック回帰モデルの定義
class RogiModel(Chain):
    def __init__(self):
        super(RogiModel, self).__init__(
            ## ロジスティック回帰：1つの線形作用のみ
            l1 = L.Linear(4,3),
        )
    
    def __call__(self, x, y):
        return F.mean_squared_error(self.fwd(x), y)
    
    def fwd(self, x):
        return F.softmax(self.l1(x))

In [6]:
## ロジスティック回帰モデルの初期化
model = RogiModel()
optimizer = optimizers.Adam()
optimizer.setup(model)

In [7]:
## ミニバッチを作成して学習開始
n = 74     # 訓練データ数
bs = 25    # バッチサイズ

for j in range(2000):
    ## 入力データをランダムに入れ換えてバッチ化
    sffindx = np.random.permutation(n)
    for i in range(0, n, bs):
        x = Variable(xtrain[sffindx[i:(i+bs) if (i+bs)<n else n]])
        y = Variable(ytrain[sffindx[i:(i+bs) if (i+bs)<n else n]])
        model.zerograds()
        loss = model(x,y)
        ## バッチサイズごとに誤差を積算する
        loss.backward()
        optimizer.update()

In [8]:
## 検証用データxtで学習結果をテスト
xt = Variable(xtest, volatile='on')
yy = model.fwd(xt)
ans = yy.data
nrow, ncol = ans.shape

In [10]:
ok = 0
for i in range(nrow):
    ## 確率が最大の結果をclsに出力
    cls = np.argmax(ans[i,:])
    print (ans[i,:], cls)
    if cls == yans[i]:
        ok += 1

print ("correct rate : ", ok, "/", nrow, "=", (ok*1.0)/nrow)

[ 0.89698702  0.07284725  0.03016579] 0
[ 0.88775039  0.07865492  0.03359475] 0
[ 0.89819521  0.07204527  0.02975949] 0
[ 0.88533634  0.08013948  0.03452419] 0
[ 0.86175078  0.09480416  0.04344503] 0
[ 0.90106237  0.07027731  0.02866029] 0
[ 0.87776858  0.08499432  0.0372371 ] 0
[ 0.91624594  0.06045221  0.02330182] 0
[ 0.90926552  0.06497484  0.02575963] 0
[ 0.89770693  0.07250272  0.02979043] 0
[ 0.88277662  0.08199131  0.03523196] 0
[ 0.90792531  0.06571873  0.02635599] 0
[ 0.85107321  0.10152733  0.04739946] 0
[ 0.87821656  0.08477569  0.03700776] 0
[ 0.89564437  0.07374116  0.03061442] 0
[ 0.86448866  0.09326675  0.04224462] 0
[ 0.90738761  0.06615687  0.02645559] 0
[ 0.87688535  0.08555975  0.03755487] 0
[ 0.90473938  0.06794052  0.02732016] 0
[ 0.8747893   0.08671153  0.03849918] 0
[ 0.89878696  0.07168563  0.02952742] 0
[ 0.88261569  0.08178024  0.03560406] 0
[ 0.87451887  0.08714495  0.03833615] 0
[ 0.89558232  0.07374429  0.03067337] 0
[ 0.89996606  0.07096536  0.02906863] 0
